# Simulations of Finite Discrete Markov Chain

This Markov chain is irreducible, if it can go from every state to every state in a finite amount of time.

In this assignment, we are interested in two properties of a finite discrete markov chian. 

1. Stationary distribution
2. Hitting time, which is the expected steps from state $i$ to state $j$

The stationary distribution for an irreducible recurrent Markov Chain is the probability distribution to which the process converges for large values of $t$

A stationary distribution $\pi$ is a (row) vector, whose entries are non-negative and sum to 1, is unchanged by the operation of transition matrix P on it and so is defined by

$$
{\displaystyle \pi \mathbf {P} =\pi .}
$$


## Analytical Class

In [87]:
import numpy as np
import scipy.linalg as LA


class finite_discrete_markov_chain():
#the initialization of the class
    def __init__(self, tr):
        # raise an error if the input is not the numpy matrix
        if str(type(tr))!="<class 'numpy.matrixlib.defmatrix.matrix'>":
            print("Error: Please enter a numpy matrix class")
            return None
        
        # check the shape of the matrix
        if len(tr)!=len(np.squeeze(np.asarray(tr[0]))): 
            print("Error: transition matrix is not shape NxN")
            return None
        
        # check if all exit having sum of probability 1
        for ticker in range(len(tr)):
            if abs(sum(np.squeeze(np.asarray(tran_mat))[ticker])-1)>0.00001:
                print("Error: total transition on a state not equal to 1")
                return None
        self.transition_matrix = tr
        self.number_of_states = len(tr)
 
    def Q_absorbing_state(self):
        absorbing_states = []
        for ticker in range(self.number_of_states):
            if self.transition_matrix[ticker,ticker] ==1: absorbing_states.append(ticker)

        if len(absorbing_states)>0: 
            print("The following are absorbing states")
            print(absorbing_states)
            return True
        else: return False
        
    def stationary_distribution(self):
        if self.Q_absorbing_state()==True:
            return "Therefore, non-absorbing states have no distributions"
        else:
            Q= np.transpose(self.transition_matrix)-np.identity(self.number_of_states)
            Before_LA = np.column_stack([Q, np.zeros(self.number_of_states)])
            Before_LA=np.row_stack([Before_LA, np.ones(self.number_of_states+1)])
            a = Before_LA[:,np.arange(0,self.number_of_states)]
            b = Before_LA[:,-1]
            return LA.solve(np.squeeze(np.asarray(a[1:len(a)+1])),
                            np.squeeze(np.asarray(b[1:len(b)+1])))

    def steps_from_state_i_to_state_j(self,state_i, state_j):
        if self.Q_absorbing_state()==True:
            return "You might want to consider the function: time to absorption. "
        else:
            print("The average number of steps needed to go from state i to state j is:")            
            if state_i==state_j:
                return 0
            
            PT = np.delete(self.transition_matrix,state_j,0)
            PT = np.delete(PT,state_j,1)
            
            return np.matmul(
                np.linalg.inv(np.eye(self.number_of_states-1)- PT),
                np.transpose(np.matrix(np.ones(self.number_of_states-1)))
            )[state_i-((state_j<state_i))].item(0)

The expected hitting  time can be calculated via:

np.transpose(np.matrix(np.ones(self.number_of_states-1)))

$$
(I_{NStaets-1} - P_T)^{-1}   \cdot C_{NStaets-1}
$$


Where

* The $I_{NStaets-1}$ denotes the identity matrix with dimension of $NStaets-1$
* $P_T$ means the transient matrix with all the transient states
* The $C_{NStaets-1}$ means  a column vector with $NStates-1$ amount of ones.  


After this calculation, we obtained a column matrix with expected numbers of steps needed to go from state $i$ to state $j$, for all $i \neq j$

Here is the analytical result of stationary distribution

In [89]:
tran_mat = np.matrix([[1/6, 5/6,0,0,0], 
                      [1/6, 1/6, 4/6, 0,0],
                      [0,1/6,2/6,3/6,0],
                      [0,0,1/6,3/6,2/6],
                      [1,0,0,0,0]
                     ])

In [90]:
chain0 = finite_discrete_markov_chain(tran_mat)

In [91]:
chain0.stationary_distribution()

array([ 0.15137615,  0.20642202,  0.27522936,  0.27522936,  0.09174312])

The above is nothing else but  the linear solution of the  ${\displaystyle \pi \mathbf {P} =\pi .}$

Here is the analytical result of hitting time

In [92]:
chain0.steps_from_state_i_to_state_j(3,4)

The average number of steps needed to go from state i to state j is:


4.299999999999999

## Numerical Class

We can naively simulate the moving of Markov Chain by randomly selection $s$ starting points and $k$ jumps

The idea of from one state going to some other states based on the probability is to set up intervals in [0,1] with length corresponding to each transition probability. The random variable following an uniform distribution in [0,1] can lie inside of different sub-intervals and tell us what next state it is going to be

In [124]:
import random
from collections import Counter
class NFDMC():
    def __init__(self, tr, start_state):        
        self.states = [_ for _ in range(len(tr))]
        self.transition_function = tr
        self.current_state = start_state
        return
    
    def find_current_state(self):
        return self.current_state
    
    def find_next_state(self):
        current_transitions = [self.transition_function[self.current_state,_] for _  in range(len(self.states))]
        
        current_intervals = []
        
        # here is to generate the intervals. 
        for i in current_transitions:
            if len(current_intervals)==0: #skip the first one
                current_intervals=[i]
                continue
            current_intervals.append(i+current_intervals[-1])
        
        rand_seed = np.random.uniform(0,1)
        
        
        
        i=0 # indicating the current state
        for threshold in current_intervals:
            if rand_seed< threshold:
                break
            else:
                i=i+1
        self.current_state = i
        return self.current_state
    
    
    def set_start_state_find_k_step_aheads_state_and_store_result(self,start_state,k):
        self.current_state = start_state
        result_list = [self.current_state]
        for i in range(k):
            result_list.append(self.find_next_state())
        return result_list
    
    
    def Nstationary_distribution(self):
        total_result = []

        n_start = 100
        n_path = 100

        for i in range(n_start):
            r_start = random.choice(self.states)
            for j in range(n_path):
                tmp = self.set_start_state_find_k_step_aheads_state_and_store_result(r_start,n_path)
                total_result+=tmp
        
        n_states =len(self.states)
        
        
        
        return np.array([Counter(total_result)[_] for _ in range(n_states)])/(len(total_result))
    
    def NHitting_time(self, start_state, end_state):
        print("WARNING, THIS METHOD WILL RESET THE STARTING STATE OF THE NDFMC CLASS")
        self.current_state = start_state
        step_list = []
        
        
        for rep in range(10000):
            self.current_state = start_state
            tmp_n_step = 0
            while(self.current_state!=end_state):
                self.current_state = self.find_next_state()
                tmp_n_step+=1
            step_list.append(tmp_n_step)

        return np.mean(step_list)

In [125]:
chain1 = NFDMC(tran_mat,1)

In [95]:
chain1.Nstationary_distribution()

array([ 0.15256634,  0.20722772,  0.27473069,  0.27336535,  0.0921099 ])

In [96]:
chain0.stationary_distribution()

array([ 0.15137615,  0.20642202,  0.27522936,  0.27522936,  0.09174312])

As you can see, those results are quite similar

In [126]:
chain1.NHitting_time(3,4)

WARNING, THIS METHOD WILL RESET THE STARTING STATE OF THE NDFMC CLASS


4.3639999999999999

In [123]:
chain0.steps_from_state_i_to_state_j(3,4)

The average number of steps needed to go from state i to state j is:


4.299999999999999

And as you can see, those two results are similar. 